# GamessFortranReader tutorial

`GamessFortranReader` is a class containing methods for reading Gamess sequential, unformatted fortran files containig

* two-electron integerals over AO's (\$JOB.F08)
* two-electron integrals over MO's (\$JOB.F09)
* two-particle density matrix (\$JOB.F15)

# IMPORTANT NOTE

As with all python iterable objects the counting starts from 0 not from 1 as in fortran. This means for example that the first element of the two electron integral supermatix **W** will be $<00|00>$ in python and not $<11|11>$ as in fortran. This of course means that the orbitals will be also indexed from 0 and all the loop over the elements of matrices and vectors should start from 0.

In [ ]:
import chemtools

In [ ]:
from chemtools.calculators.gamessreader import GamessFortranReader, print_twoe, ijkl, factor
from chemtools.calculators.gamessus import GamessUS
import numpy as np
import os

First we need to generate some files to work on. Below I'm using my own gamess installation with custom `rungamessave` script which compies the .F?? (with integrals etc.) files to my current directory after the job is finished. 

In [ ]:
gamess = Gamess(executable="/home/lmentel/Programs/gamess-us-dec2014/rungmssave",
                version="00",
                runopts=None,
                scratch="/home/lmentel/scratch")

As an example I will use H$_2$ from earlier. Here's the input

In [ ]:
inpstr = """ $CONTRL scftyp=rhf runtyp=energy maxit=30 mult=1 ispher=1
     itol=30 icut=30 units=bohr cityp=guga qmttol=1.0e-8 $END
 $SYSTEM timlim=525600 mwords=100 $END
 $SCF dirscf=.false. $END
 $CIINP
    nrnfg(6)=1
 $END
 $CIDRT iexcit=2 nfzc=0 ndoc=1 nval=27 group=d2h stsym=ag
        mxnint=14307305 $END
 $GUGDIA prttol=1.0e-6 cvgtol=1.0e-10 $END
 $DATA
H2 cc-pVTZ
dnh 2

H    1.00       0.000000       0.000000       0.700000
S   3
  1     33.8700000              0.0060680        
  2      5.0950000              0.0453080        
  3      1.1590000              0.2028220        
S   1
  1      0.3258000              1.0000000        
S   1
  1      0.1027000              1.0000000        
P   1
  1      1.4070000              1.0000000        
P   1
  1      0.3880000              1.0000000        
D   1
  1      1.0570000              1.0000000        

 $END
"""

I'll create a `temp` folder as my workdir for the calculation

In [ ]:
os.mkdir('temp')
os.chdir('temp')
os.getcwd()

Now I can write the input file

In [ ]:
inpfile = "h2_eq_pvtz_fci.inp"
with open(inpfile, 'w') as inp:
    inp.write(inpstr)

and run the calculation

In [ ]:
logfile = gamess.run(inpfile)

In [ ]:
!ls

All the important files are there. Now I can create the `GamessFortranReader` giving it the `logfile` (name out the output) as an argument since it need to figure out how many AO's and MO's there are before reading the integral files.

## reading the two-electron integrals in AO

In [ ]:
gfr = GamessFortranReader(logfile)

In [ ]:
twoeAO = gfr.read_twoeao()
twoeAO

In [ ]:
nao = gfr.gp.get_number_of_aos()
print_twoe(twoeAO, nao)

## reading the two-electron integrals in MO

In [ ]:
twoeMO = gfr.read_twoemo()
twoeMO

In [ ]:
nmo = gfr.gp.get_number_of_mos()
print_twoe(twoeMO, nmo)

## reading the two particle density matrix

In [ ]:
twoRDM = gfr.read_rdm2()
twoRDM

In [ ]:
print_twoe(twoRDM, nmo)

## Calculate the two-electron energy from the two-electron integrals and 2RDM

In [ ]:
energy = 0.0

# loop over elements and sum them
ij = 0
for i in xrange(nmo):
    for j in xrange(i+1):
        ij += 1
        kl = 0
        for k in xrange(nmo):
            for l in xrange(k+1):
                kl += 1
                if ij >= kl:
                    energy += factor(i,j,k,l)*twoeMO[ijkl(i,j,k,l)]*twoRDM[ijkl(i,j,k,l)]
                    #print(i,j,k,l,factor(i,j,k,l), twoeint[ijkl(i,j,k,l)], twordm[ijkl(i,j,k,l)])

print("Two-electron energy: ", 0.5*energy)

In [ ]:
gfr.gp.get_energy_components('ci')

As you can see the energy obtained by contracting twoMO with twoRDM ('Two-electron energy: ', 0.58866020149518394) is the same as the one parsed from the logfile ('ELECTRON-ELECTRON POTENTIAL ENERGY': 0.5886602014,).